# Пример использования библиотеки BigARTM для тематического моделирования

Импортируем BigARTM:

In [23]:
from matplotlib import pyplot as plt
%matplotlib inline

In [24]:
import artm

Первое считывание данных (преобразуем удобный для человека формат в формат, который использует модель):

In [29]:
batch_vectorizer = artm.BatchVectorizer(data_path='school.txt', data_format='vowpal_wabbit', 
                                        target_folder='school_batches', batch_size=100,
                                       gather_dictionary=True)

В следующий раз данные можно считывать уже из батчей:

In [30]:
batch_vectorizer = artm.BatchVectorizer(data_path='school_batches', data_format='batches',
                                       gather_dictionary=True)

Создаем объект модели:

In [31]:
T = 10 # количество тем
model_artm = artm.ARTM(num_topics=T, topic_names=['sbj'+str(i) for i in range(T)], class_ids={'text':1})
# число после названия модальностей - это их веса

In [16]:
# works only by one
model_artm.scores.add(artm.PerplexityScore(name='PerplexityScore'))
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore', class_id="text"))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopTokensScore(name="top_words", num_tokens=15, class_id="text"))

Создаем словарь и инициализируем модель с его помощью

In [32]:
# doesn't work
model_artm.gather_dictionary('dictionary', batch_vectorizer.data_path)

AttributeError: 'ARTM' object has no attribute 'gather_dictionary'

Строим модель. Offline - проходит по всей коллекции много раз. Удобно, когда коллекция маленькая.

Необходимое число итераций можно отслеживать по графику перплексии. Когда она перестала меняться, модель сошлась.

Выведем топы слов:

Давайте посмотрим также на разреженность матриц:

В темах много общеупотребительных слов (так называемой, фоновой лексики). Чтобы этого избежать, будем использовать разреживающий регуляризатор для матрицы фи. Он будет подавлять слова, которые имеют большую частоту во всей коллекции.

In [ ]:

#если вы хотите применять регуляризатор только к некоторым модальностям, указывайте это в параметре class_ids: class_ids=["text"]

Попробуем менять коэффициент регуляризации:

In [ ]:
# еще раз посмотрим на разреженность


Обратите внимание, что разреживать модель рекомендуется только после того, как она сошлась без регуляризации.

Сохранение и загрузка модели:

Можно попросить матрицы в чистом виде:

Матрица вероятностей тем в документах.

Если бы у нас были новые батчи, по которым мы не строим модель, а хотим только получить матрицу theta, можно пользоваться методом transform.